# ⚽️ 축구 시맨틱 세그멘테이션 증강 파이프라인 분석 및 추천

제공된 `GetTransfrom(type)` 함수와 추천된 **Type "D" (최강 증강)**를 포함하여, 데이터 증강 전략을 A, B, C, D 네 가지 유형으로 분석하고 축구 시맨틱 세그멘테이션 데이터셋에 가장 적합한 방법을 제안.

---

## A. 기본 증강

가장 **일반적이고 필수적인** 증강 및 전처리 단계.

| 순서 | 변환 (Transform) | 목적 | 상세 내용 |
| :---: | :--- | :--- | :--- |
| 1 | `v2.Resize((256, 256))` | **크기 조정** | 모든 이미지를 256x256 픽셀로 조정합니다. |
| 2 | `v2.RandomHorizontalFlip(p=0.5)` | **좌우 반전** | 50% 확률로 수평으로 뒤집습니다. |
| 3 | `v2.ColorJitter(...)` | **색상 변화** | 밝기, 대비, 채도, 색조를 무작위로 변경합니다. |
| 4 | `v2.ToTensor()` | **Tensor 변환** | 이미지를 PyTorch Tensor로 변환 및 `[0, 1]` 정규화합니다. |
| 5 | `v2.Normalize(...)` | **정규화** | ImageNet 표준값으로 표준화합니다. |


## B. 강력 증강

A 유형에 **기하학적 및 노이즈 증강**을 추가하여 모델의 **강건성** 향상.

| 순서 | 변환 (Transform) | 목적 | 상세 내용 |
| :---: | :--- | :--- | :--- |
| 1-3 | `v2.Resize`, `Flip`, `Jitter` | **기본 증강** | A 유형과 동일한 기본 변환 적용 |
| 4 | `v2.RandomRotation(degrees=15)` | **무작위 회전** | $\pm 15$도 범위 내에서 무작위 회전 (**추가**) |
| 5 | `v2.RandomAffine(...)` | **어파인 변환** | 최대 10% 이동 및 90%~110% 크기 조정 (**추가**) |
| 6 | `v2.GaussianBlur(...)` | **가우시안 블러** | 무작위 블러 적용 (**추가**) |
| 7 | `v2.RandomErasing(...)` | **무작위 소거** | 30% 확률로 이미지의 일부 영역 가림 (**추가**) |
| 8-9 | `v2.ToTensor`, `v2.Normalize` | **전처리** | A 유형과 동일한 전처리 적용 |


## C. Segmentation 최적화 증강 (원근 추가)

B 유형에서 **Resize**를 제거하고 **시맨틱 세그멘테이션에 중요한 원근 변환 (`RandomPerspective`)**을 추가.

| 순서 | 변환 (Transform) | 변경/추가 사항 | 목적 |
| :---: | :--- | :--- | :--- |
| 1 | `v2.RandomResizedCrop(...)` | `v2.Resize` **대체**. | **스케일 및 부분 영역 학습** 유도 |
| 2-4 | `v2.RandomHorizontalFlip` 외 2종 | B 유형 유지. | |
| 5 | **`v2.RandomPerspective(...)`** | **핵심 추가**. | **다양한 카메라 시점 및 원근 왜곡** 학습 |
| 6-8 | `v2.ColorJitter` 외 2종 | B 유형 유지. | |
| 9-10 | `v2.ToTensor`, `v2.Normalize` | B 유형 유지. | |


## D. 최강 증강 (C + 화질 변화 대응)

C 유형에 **`v2.RandomAdjustSharpness`**를 추가하여 다양한 영상/사진 화질 변화에 대한 모델의 **강건성**을 확보한 최종 추천 파이프라인.

| 순서 | 변환 (Transform) | 변경/추가 사항 | 목적 |
| :---: | :--- | :--- | :--- |
| 1-5 | `v2.RandomResizedCrop` ~ `v2.RandomPerspective` | C 유형 기하학적 증강 유지. | **강력한 시점 및 스케일 학습** |
| 6-7 | `v2.ColorJitter`, `v2.GaussianBlur` | C 유형 유지. | |
| 8 | **`v2.RandomAdjustSharpness(...)`** | **화질 변화 대응 추가**. | 이미지의 **선명도**를 무작위로 변경하여 강건성 확보 |
| 9 | `v2.RandomErasing(...)` | C 유형 유지. | |
| 10-11 | `v2.ToTensor`, `v2.Normalize` | C 유형 유지. | **필수 전처리** |


## ✅ 결론 및 권장 사항

축구 시맨틱 세그멘테이션 작업은 **픽셀 단위의 정확도**와 **다양한 촬영 환경**에 대한 일반화가 중요하다.

**Type "D"**는 **`RandomPerspective`**를 통한 강력한 시점 변화 학습과 **`RandomAdjustSharpness`**를 통한 화질 변화 대응 능력을 모두 갖추고 있어, 해당 데이터셋에 대한 학습 성능을 가장 극대화할 수 있는 **최종 추천 파이프라인**이라고 생각 된다.(A,D로 테스트할 예정)

### UNet
### 1. 특징 맵 채널 수 (Channel) 변화 (표준 UNet)

| 단계 | 이전 레이어 출력 채널 | 다음 레이어 입력 채널 | 다음 레이어 출력 채널 | 주요 동작 |
| :--- | :--- | :--- | :--- | :--- |
| **Encoder1** | 3 (RGB) | 3 | 64 | 초기 특징 추출 |
| **Encoder2** | 64 | 64 | 128 | 다운샘플링 + 특징 추출 |
| **Encoder3** | 128 | 128 | 256 | 다운샘플링 + 특징 추출 |
| **Bottleneck** | 256 | 256 | 512 | 최저 해상도 특징 추출 |
| **Decoder3** | 512 (Bottleneck) | 512 | 256 | UpConv(256) + Encoder3(256) 결합 |
| **Decoder2** | 256 (Decoder3) | 256 | 128 | UpConv(128) + Encoder2(128) 결합 |
| **Decoder1** | 128 (Decoder2) | 128 | 64 | UpConv(64) + Encoder1(64) 결합 |
| **Final Conv** | 64 (Decoder1) | 64 | 11 | 최종 클래스 분류 (로짓) |

### 2. 특징 맵 해상도 (Resolution) 변화 (표준 UNet)

| 단계 | 채널 수 변화 | 해상도 변화 (예: $H \times W$) | 동작 |
| :--- | :--- | :--- | :--- |
| **Encoder 1** | $3 \to 64$ | $H \times W$ | Conv Block |
| **Encoder 2** | $64 \to 128$ | $H/2 \times W/2$ | MaxPool (다운샘플링) |
| **Encoder 3** | $128 \to 256$ | $H/4 \times W/4$ | MaxPool (다운샘플링) |
| **Bottleneck** | $256 \to 512$ | $H/8 \times W/8$ | MaxPool + Conv Block |
| **Decoder 3** | $512 \to 256$ | $H/4 \times W/4$ | UpConv (해상도 2배 상승) |
| **Decoder 2** | $256 \to 128$ | $H/2 \times W/2$ | UpConv (해상도 2배 상승) |
| **Decoder 1** | $128 \to 64$ | $H \times W$ | UpConv (해상도 2배 상승) |
| **Final Conv** | $64 \to 11$ | $H \times W$ | 클래스 예측 |


# AdvancedUNet Class

1. 특징 맵 채널 수 (Channel) 변화

이 표는 네트워크의 각 주요 단계에서 특징 맵의 깊이(depth)인 **채널 수**가 어떻게 변하는지를 보여줍니다. 디코더 단계의 입력은 **업샘플링**된 특징과 **어텐션이 적용된 인코더 특징**이 결합된 후의 채널 수입니다.

| 단계 | 이전 레이어 출력 채널 | 다음 레이어 입력 채널 | 다음 레이어 출력 채널 | 주요 동작 |
| :--- | :--- | :--- | :--- | :--- |
| **Encoder1** | 3 (RGB) | 3 | 64 | 초기 특징 추출 |
| **Encoder2** | 64 | 64 | 128 | 다운샘플링 + 특징 추출 |
| **Encoder3** | 128 | 128 | 256 | 다운샘플링 + 특징 추출 |
| **Bottleneck** | 256 | 256 | 512 | 최저 해상도 특징 추출 |
| **Decoder3** | 512 (Bottleneck) | 512 | 256 | UpConv(256) + Attn(256) 결합 |
| **Decoder2** | 256 (Decoder3) | 256 | 128 | UpConv(128) + Attn(128) 결합 |
| **Decoder1** | 128 (Decoder2) | 128 | 64 | UpConv(64) + Attn(64) 결합 |
| **Final Conv** | 64 (Decoder1) | 64 | 11 | 최종 클래스 분류 (로짓) |

2. 특징 맵 해상도 (Resolution) 변화

이 표는 입력 해상도를 $H \times W$라고 가정했을 때, **Pooling(다운샘플링)**과 **UpConv(업샘플링)**에 따라 해상도가 어떻게 변하는지를 보여줍니다.

| 단계 | 채널 수 변화 | 해상도 변화 (예: $H \times W$) | 동작 |
| :--- | :--- | :--- | :--- |
| **Encoder 1** | $3 \to 64$ | $H \times W$ | Conv Block |
| **Encoder 2** | $64 \to 128$ | $H/2 \times W/2$ | MaxPool (다운샘플링) |
| **Encoder 3** | $128 \to 256$ | $H/4 \times W/4$ | MaxPool (다운샘플링) |
| **Bottleneck** | $256 \to 512$ | $H/8 \times W/8$ | MaxPool + Conv Block |
| **Decoder 3** | $512 \to 256$ | $H/4 \times W/4$ | UpConv (해상도 2배 상승) |
| **Decoder 2** | $256 \to 128$ | $H/2 \times W/2$ | UpConv (해상도 2배 상승) |
| **Decoder 1** | $128 \to 64$ | $H \times W$ | UpConv (해상도 2배 상승) |
| **Final Conv** | $64 \to 11$ | $H \times W$ | 클래스 예측 |

### 2. 특징 맵 해상도 (Resolution) 변화

이 표는 입력 해상도를 $H \times W$라고 가정했을 때, **Pooling(다운샘플링)**과 **UpConv(업샘플링)**에 따라 해상도가 어떻게 변하는지를 보여줍니다.

| 단계 | 채널 수 변화 | 해상도 변화 (예: $H \times W$) | 동작 |
| :--- | :--- | :--- | :--- |
| **Encoder 1** | $3 \to 64$ | $H \times W$ | Conv Block |
| **Encoder 2** | $64 \to 128$ | $H/2 \times W/2$ | MaxPool (다운샘플링) |
| **Encoder 3** | $128 \to 256$ | $H/4 \times W/4$ | MaxPool (다운샘플링) |
| **Bottleneck** | $256 \to 512$ | $H/8 \times W/8$ | MaxPool + Conv Block |
| **Decoder 3** | $512 \to 256$ | $H/4 \times W/4$ | UpConv (해상도 2배 상승) |
| **Decoder 2** | $256 \to 128$ | $H/2 \times W/2$ | UpConv (해상도 2배 상승) |
| **Decoder 1** | $128 \to 64$ | $H \times W$ | UpConv (해상도 2배 상승) |
| **Final Conv** | $64 \to 11$ | $H \times W$ | 클래스 예측 |

### TransferLearningUNet
### 1. ⚙️ 모델 구성 요소 (Initialization: `__init__`)

| 구성 요소 | 역할 | 원본 모듈 (ResNet-34) | 출력 채널 크기 |
| :--- | :--- | :--- | :--- |
| **Encoder1 (e1)** | 초기 특징 추출 | `conv1`, `bn1`, `relu` | 64 |
| **Pool** | 다운샘플링 (Max Pooling) | `maxpool` | - |
| **Encoder2 (e2)** | Layer 1 특징 추출 | `layer1` | 64 |
| **Encoder3 (e3)** | Layer 2 특징 추출 | `layer2` | 128 |
| **Encoder4 (e4)** | Layer 3 특징 추출 | `layer3` | 256 |
| **Bottleneck (b)** | 최하위 특징 추출 | `layer4` | 512 |
| **Upconv** | 디코딩을 위한 **업샘플링** (전치 컨볼루션) | `ConvTranspose2d` | 512 $\rightarrow$ 256 $\rightarrow$ 128 $\rightarrow$ 64 |
| **Decoder** | 디코더 단계의 특징 추출 및 정제 | `Conv2d` $\rightarrow$ `ReLU` | 병합된 채널 수의 $\rightarrow$ 절반 |
| **Final Conv** | 최종 출력 레이어 | `Conv2d` (커널 1) | `num_classes` |

### 2. 🌊 순전파 과정 (Forward Pass: `forward`)

| 단계 | 입력 | 모듈/작동 | 출력 변수 | 특징 맵 크기 (예시) | 설명 |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **인코딩 1** | $x$ | `encoder1` | $e1$ | $H/2 \times W/2$ | 초기 특징 추출. |
| **인코딩 2** | `pool(e1)` | `encoder2` | $e2$ | $H/4 \times W/4$ | 첫 번째 다운샘플링 및 특징 추출. **(Skip Connection 대상)** |
| **인코딩 3** | $e2$ | `encoder3` | $e3$ | $H/8 \times W/8$ | 두 번째 다운샘플링 및 특징 추출. **(Skip Connection 대상)** |
| **인코딩 4** | $e3$ | `encoder4` | $e4$ | $H/16 \times W/16$ | 세 번째 다운샘플링 및 특징 추출. **(Skip Connection 대상)** |
| **병목** | $e4$ | `bottleneck` | $b$ | $H/32 \times W/32$ | 가장 압축된 특징. |
| **디코딩 4** | $b$ | `upconv4` | $d4$ | $H/16 \times W/16$ | $b$를 2배 업샘플링. |
| | $d4, e4$ | `torch.cat((d4, e4), dim=1)` | $d4$ | $H/16 \times W/16$ | **Skip Connection**: $e4$와 채널 방향으로 병합 (Concatenation). |
| | $d4$ | `decoder4` | $d4$ | $H/16 \times W/16$ | 병합된 특징을 정제. |
| **디코딩 3** | $d4$ | `upconv3` | $d3$ | $H/8 \times W/8$ | $d4$를 2배 업샘플링. |
| | $d3, e3$ | `torch.cat((d3, e3), dim=1)` | $d3$ | $H/8 \times W/8$ | **Skip Connection**: $e3$와 병합. |
| | $d3$ | `decoder3` | $d3$ | $H/8 \times W/8$ | 병합된 특징을 정제. |
| **디코딩 2** | $d3$ | `upconv2` | $d2$ | $H/4 \times W/4$ | $d3$를 2배 업샘플링. |
| | $d2, e2$ | `torch.cat((d2, e2), dim=1)` | $d2$ | $H/4 \times W/4$ | **Skip Connection**: $e2$와 병합. |
| | $d2$ | `decoder2` | $d2$ | $H/4 \times W/4$ | 병합된 특징을 정제. |
| **디코딩 1** | $d2$ | `upconv1` | $d1$ | $H/2 \times W/2$ | $d2$를 2배 업샘플링. |
| | $d1, e1$ | `torch.cat((d1, e1), dim=1)` | $d1$ | $H/2 \times W/2$ | **Skip Connection**: $e1$과 병합. |
| | $d1$ | `decoder1` | $d1$ | $H/2 \times W/2$ | 병합된 특징을 정제. |
| **최종 출력** | $d1$ | `nn.functional.interpolate` | $out$ | $H \times W$ | **최종 업샘플링** (원본 크기로 복원). |
| | $out$ | `final_conv` | `return` | $H \times W$ | 최종 클래스 예측 (Channel: `num_classes`). |